In [12]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
using Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
    pkg"add FIB#master"
end
Pkg.instantiate()
Pkg.build("Cairo")
#---

   Cloning git-repo `https://github.com/JuliaPOMDP/FIB.jl.git`
  Updating git-repo `https://github.com/JuliaPOMDP/FIB.jl.git`.4 %
 Resolving package versions...
 Installed Tables ───── v0.1.12
 Installed DataFrames ─ v0.15.0
  Updating `~/Kapow/Docs/Stanford/Coterm/CS238/Final Project/CS238-final/Project.toml`
  [13b007ba] ↑ FIB v0.2.0 ⇒ v0.3.0 #master (https://github.com/JuliaPOMDP/FIB.jl.git)
  Updating `~/Kapow/Docs/Stanford/Coterm/CS238/Final Project/CS238-final/Manifest.toml`
  [a93c6f00] ↑ DataFrames v0.14.1 ⇒ v0.15.0
  [13b007ba] ↑ FIB v0.2.0 ⇒ v0.3.0 #master (https://github.com/JuliaPOMDP/FIB.jl.git)
  [bd369af6] ↑ Tables v0.1.11 ⇒ v0.1.12
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
Fetching: [========================================>]  100.0 %.0 %  Building LibCURL ─→ `~/.julia/package

In [13]:
# import necessary packages
using AA228FinalProject
using TabularTDLearning
using POMDPs
using FIB # For the solver
using QMDP
using POMDPModels
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf

In [14]:
sensor = Bumper()
config = 3 # 1,2, or 3
ds = DiscreteRoombaStateSpace(50, 50, 20)
v_steps = [2,5]
om_steps = [-3*pi/4, -pi/2, -pi/4, 0.0, pi/4, pi/2, 3*pi/4, pi]

aspace = vec(collect(RoombaAct(v,om) for v in v_steps, om in om_steps))
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(sspace=ds, aspace=aspace, config=config))


RoombaPOMDP{Bumper,Bool}(Bumper(), RoombaMDP{DiscreteRoombaStateSpace,Array{RoombaAct,1}}
  v_max: Float64 10.0
  om_max: Float64 1.0
  dt: Float64 0.5
  contact_pen: Float64 -1.0
  time_pen: Float64 -0.1
  goal_reward: Float64 10.0
  stairs_penalty: Float64 -10.0
  config: Int64 3
  room: AA228FinalProject.Room
  sspace: DiscreteRoombaStateSpace
  aspace: Array{RoombaAct}((16,))
  _amap: Dict{RoombaAct,Int64}
)

In [15]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

solver = FIBSolver(max_iterations=2, tolerance=1e-3, verbose=true)
policy = solve(solver, m)

MethodError: MethodError: no method matching reward(::RoombaPOMDP{Bumper,Bool}, ::RoombaState, ::RoombaAct)
Closest candidates are:
  reward(::Union{RoombaMDP, RoombaPOMDP}, ::AbstractArray{Float64,1}, ::AbstractArray{Float64,1}, !Matched::AbstractArray{Float64,1}) at /Users/sarahradz/Kapow/Docs/Stanford/Coterm/CS238/Final Project/CS238-final/src/roomba_env.jl:339
  reward(::Union{MDP, POMDP}, ::Any, ::Any, !Matched::Any) at /Users/sarahradz/.julia/packages/POMDPs/JiYXY/src/pomdp.jl:107
  reward(!Matched::POMDPModelTools.FullyObservablePOMDP{S,A}, ::S, ::A) where {S, A} at /Users/sarahradz/.julia/packages/POMDPModelTools/eHEjm/src/fully_observable_pomdp.jl:41
  ...

In [ ]:
Pkg.build("HDF5")
Pkg.add("JLD")
using JLD

In [ ]:
JLD.save("qmdp_policy.jld", "policy", policy)

In [ ]:
@load "qmdp_policy.jld"

In [11]:
# first seed the environment
Random.seed!(5)

# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, policy, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

MethodError: MethodError: no method matching stateindex(::RoombaState)
Closest candidates are:
  stateindex(!Matched::POMDPModelTools.FullyObservablePOMDP{S,A}, !Matched::S) where {S, A} at /Users/heathershen/.julia/packages/POMDPModelTools/eHEjm/src/fully_observable_pomdp.jl:46
  stateindex(!Matched::POMDPModelTools.UnderlyingMDP{P,Int64,A}, !Matched::Int64) where {P, A} at /Users/heathershen/.julia/packages/POMDPModelTools/eHEjm/src/underlying_mdp.jl:29
  stateindex(!Matched::POMDPModelTools.UnderlyingMDP{P,Bool,A}, !Matched::Bool) where {P, A} at /Users/heathershen/.julia/packages/POMDPModelTools/eHEjm/src/underlying_mdp.jl:30
  ...